In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
locations_isochrones = dataiku.Dataset("locations_isochrones")
locations_isochrones_df = locations_isochrones.get_dataframe()

In [0]:
project_key = dataiku.get_custom_variables()["projectKey"]
client = dataiku.api_client()
project = client.get_project(project_key)
variables = project.get_variables()["standard"]
isochrones_api_to_use = variables["isochrones_api_to_use"]
variables_isochrones_api = variables[isochrones_api_to_use]

In [0]:
points_of_interest_tags = variables["points_of_interest_tags"]
isochrones_range_min = variables["isochrones_range_min"]
isochrones_range_min = ["isochrone_{}_min".format(isochrone) \
                                     for isochrone in isochrones_range_min]

In [0]:
from ast import literal_eval

In [0]:
import logging
import requests

from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

In [0]:
import time

In [0]:
def _check_tags(tags):
    if tags is None:
        raise ValueError("Tags not specified")

In [0]:
def collect_points_of_interest_in_envelope(request_session, envelope_str, tags):
    # Modified based on https://stackoverflow.com/questions/23267409/how-to-implement-retry-mechanism-into-python-requests-library
    overpass_url = "http://overpass-api.de/api/interpreter"
    _check_tags(tags)
    tags = [t.strip().lower() for t in tags]
    tags = [t for t in tags if len(t) > 0]
    tags = set(tags) # deduplicate
    overpass_query = "[out:json];("
    if len(tags) > 0:
        for t in tags:
            overpass_query += 'node'+'['+t+']'+envelope_str+';'+'way'+'['+t+']'+envelope_str+';'
        overpass_query = overpass_query+')'+';out center;'
    else:
        overpass_query = overpass_query[:-1]+envelope_str+';out center;'

    response = request_session.get(overpass_url, params={'data': overpass_query})
    response_status_code = response.status_code
    try:

        data = response.json()
    except Exception as e :
        print("Exception met! Query is : '{}' | Exceptions args are : '{}' | output response will be empty"\
              .format(overpass_query, e.args))
        data = {}
    return data, response_status_code

In [0]:
locations_isochrones_df.head(3)

In [0]:
logging.basicConfig(level=logging.DEBUG)
request_session = requests.Session()
retries = Retry(total=10, backoff_factor=1, status_forcelist=[502, 503, 504, 429])
request_session.mount('http://', HTTPAdapter(max_retries=retries))

In [0]:
isochrones_points_of_interest = {}
missed_points_of_interest = {}
start_time = time.time()
for isochrone_type in isochrones_range_min:
    missed_points_of_interest[isochrone_type] = []
    isochrones_points_of_interest[isochrone_type] = []
    focus_isochrones = list(locations_isochrones_df[isochrone_type])
    focus_isochrones = [literal_eval(isochrone) for isochrone in focus_isochrones]
    n_isochrones = len(focus_isochrones)

    for isochrone_index, isochrone in enumerate(focus_isochrones):

        isochrone_envelope = isochrone["properties"]["envelope_str"]
        points_of_interest_data, response_status_code = collect_points_of_interest_in_envelope(request_session,
                                                                                               isochrone_envelope,
                                                                                               points_of_interest_tags)
        response_status_label = "OK" if response_status_code == 200 else "KO"
        spent_time = time.time() - start_time

        average_collection_time =  spent_time/(isochrone_index + 1.0)
        estimated_time_to_finish = (n_isochrones - (isochrone_index + 1.0)) * average_collection_time
        print("""\nIsochrone type : '{}' | Isochrone index : {} (total of {})"""\
              """Isochrone_envelope : '{}' | Response code : '{}' ({}) | spent time : {} seconds \n"""\
              .format(isochrone_type,
                      isochrone_index,
                      n_isochrones,
                      isochrone_envelope,
                      response_status_code,
                      response_status_label,
                      spent_time))
        print("Estimated time to finish the collection (with isochrone type = '{}') : {}".format(isochrone_type,
                                                                                                 estimated_time_to_finish))

        isochrones_points_of_interest[isochrone_type].append(points_of_interest_data)
        if response_status_code != 200:
            missed_data = {"isochrone_index":isochrone_index, "isochrone_envelope": isochrone_envelope}
            missed_points_of_interest[isochrone_type].append(missed_data)

In [0]:
def from_poi_collection_to_pandas(poi_collection, poi_belonging_location):
    df_poi_collection = pd.DataFrame(poi_collection)
    df_poi_collection.rename({"id":"poi_id", "lat":"poi_lat", "lon":"poi_lon",
                              "center":"poi_center", "nodes":"poi_nodes", "type":"poi_type"},
                             axis=1, inplace=True)
    new_lats = []
    new_lons = []
    geo_points = []

    for poi_center, latitude, longitude in \
    zip(df_poi_collection["poi_center"], df_poi_collection["poi_lat"], df_poi_collection["poi_lon"]):
        if pd.isnull(poi_center):
            latitude_to_add = latitude
            longitude_to_add = longitude
        else:
            latitude_to_add = poi_center["lat"]
            longitude_to_add = poi_center["lon"]
        new_lats.append(latitude_to_add)
        new_lons.append(longitude_to_add)
        geo_points.append("POINT({} {})".format(longitude_to_add, latitude_to_add))

    df_poi_collection["poi_lat"] = new_lats
    df_poi_collection["poi_lon"] = new_lons
    df_poi_collection["poi_geo_point"] = geo_points
    df_poi_collection["poi_belonging_location"] = poi_belonging_location
    return df_poi_collection

In [0]:
df_points_of_interest = pd.DataFrame(columns=["poi_belonging_location",
                                              "poi_id",
                                              "isochrone_type",
                                              "poi_type",
                                              "poi_center",
                                              "poi_lat",
                                              "poi_lon",
                                              "poi_geo_point",
                                              "tags",
                                              "poi_nodes"])

In [0]:
locations = list(locations_isochrones_df["location_identifier"])
for isochrone_type in isochrones_range_min:
    for location, poi_collection in zip(locations, isochrones_points_of_interest[isochrone_type]):
        poi_collection = poi_collection["elements"]
        print("Extracting Isochrone '{}' points of interest ...".format(location))
        df_tmp = from_poi_collection_to_pandas(poi_collection, location)
        df_tmp["isochrone_type"] = isochrone_type
        df_points_of_interest = df_points_of_interest.append(df_tmp)

In [0]:
#We filter the points of interest dataframe only on rows that correspond to nodes (types of nodes are "node" and "way") :
FILTER_POINTS_OF_INTEREST = True
if FILTER_POINTS_OF_INTEREST :
    df_points_of_interest = df_points_of_interest[df_points_of_interest["poi_type"]=="node"]

In [0]:
final_schema = ["poi_belonging_location"]+[col for col in df_points_of_interest.columns if col != "poi_belonging_location"]
df_points_of_interest = df_points_of_interest[final_schema]

In [0]:
# Write recipe outputs
isochrones_envelopes_points_of_interest = dataiku.Dataset("isochrones_envelopes_points_of_interest")
isochrones_envelopes_points_of_interest.write_with_schema(df_points_of_interest)